<a href="https://colab.research.google.com/github/Swathi1309/Medical-Image-Analysis/blob/main/ED18B034_ED6001_Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing libraries

In [ ]:
import cv2
from skimage import filters, feature

import numpy as np
from math import log10, sqrt, floor, ceil

from matplotlib import pyplot as plt
from google.colab.patches import cv2_imshow

## Supporting funtions

In [71]:
# Function to quantify image performance by using Peak Signal to Noise Ratio
def psnr(noisy, reconstructed):
  mse = np.mean((noisy - reconstructed) ** 2)
  if mse==0:
    return 100
  max_pixel = 255
  psnr = 20*log10((max_pixel)/sqrt(mse))
  return psnr

In [72]:
# Function to denoise image using gaussian and mean filters
def filter(image, blur_type, window_size):
  if blur_type=="gaussian":
    filtered = cv2.GaussianBlur(image, (window_size, window_size), 0)
  elif blur_type=="mean":
    filtered = cv2.blur(image, (window_size, window_size))
  residual = image-filtered
  performance = psnr(image, filtered)

  image_name = blur_type+"_"+str(window_size)+".png"
  residual_name = "res"+"_"+image_name
  cv2.imwrite(image_name, filtered)
  cv2.imwrite(residual_name, residual)
  print (image_name, ", PSNR =", performance)

In [73]:
# Function to obtain PDF and CDF of an image
def histograms(image, name):
  hist = [0 for _ in range(int(np.max(image))+1)]
  cdf = [0 for _ in range(len(hist))]
  cdf[0] = hist[0]

  for i in range(image.shape[0]):
    for j in range(image.shape[1]):
      intensity = floor(image[i,j])
      hist[intensity] +=1

  for i in range(1, len(cdf)):
    cdf[i] =cdf[i-1] +hist[i]

  fig, ax = plt.subplots(1,3, figsize=(30,10))
  ax[0].imshow(image, cmap='gray')
  ax[1].bar(range(len(hist)), hist, width=0.5)
  ax[1].set_title('PDF')
  ax[2].bar(range(len(hist)), cdf, width=0.5)
  ax[2].set_title('CDF')

  plt.savefig(name)

## Edge detectors

In [74]:
def robert_edge_detector(image, name):
  edge_roberts = filters.roberts(eq);
  fig, ax = plt.subplots(1,1, figsize=(20,20));
  ax.imshow(edge_roberts, cmap='gray');
  plt.savefig(name)

In [75]:
def sobel_edge_detector(image, k, name):
  sobelx = cv2.Sobel(src=image, ddepth=cv2.CV_64F, dx=1, dy=0, ksize=k)
  sobely = cv2.Sobel(src=image, ddepth=cv2.CV_64F, dx=0, dy=1, ksize=k)
  sobelxy = cv2.Sobel(src=image, ddepth=cv2.CV_64F, dx=1, dy=1, ksize=k)
  cv2.imwrite((name+"_x.png"), sobelx);
  cv2.imwrite((name+"_y.png"), sobely);
  cv2.imwrite((name+"_xy.png"), sobelxy);

In [76]:
def canny_edge_detector(image, low, high, name):
  canny = cv2.Canny(image, low, high)
  name = name + "_" + str(low) + "_" + str(high) + ".png"
  cv2.imwrite(name, canny)

In [77]:
def laplacian_edge_detector(image, k, name):
  lap=image
  lap = cv2.Laplacian(lap, cv2.CV_16S, ksize=k)
  lap = cv2.convertScaleAbs(lap)
  name = name + "_" + str(k) + ".png"
  cv2.imwrite(name, lap)

In [78]:
def diff_of_gaussian(image, low, high, name):
  low_sigma = cv2.GaussianBlur(image,(low,low),0)
  high_sigma = cv2.GaussianBlur(image,(high,high),0)
  dog = low_sigma - high_sigma
  name = name+"_"+str(low)+"_"+str(high)+".png"
  cv2.imwrite(name, dog)

## Hough Transform

In [122]:
def hough_transform(image, m, threshold, name):
  (M, N) = image.shape
  center_x = M/2
  center_y = N/2
  a_max = M/2
  b_max = N/2
  accumulator = np.zeros((ceil(a_max), ceil(b_max)))
  for x in range(M):
    for y in range(N):
      if image[x,y]>=200:
        x_coord = x-center_x if center_x<=x else x-center_x
        y_coord = y-center_y if center_y<=y else y-center_y
        for a in range(1, ceil(a_max)):
          for b in range(1, ceil(b_max)):
            value = (x_coord/a)**m + (y_coord/b)**m
            if value>=1-threshold and value<=1+threshold:
              accumulator[a,b] +=1
  optimal = (np.where(accumulator == np.amax(accumulator)))
  a, b = optimal[0][0], optimal[1][0]

  t = np.linspace(0, 2 * np.pi, 100)
  y = ((np.abs(np.cos(t))) ** (2 / m)) * b * np.sign(np.cos(t))
  x = ((np.abs(np.sin(t))) ** (2 / m)) * a * np.sign(np.sin(t))

  plt.axis('equal')
  plt.imshow(image, cmap='gray')
  plt.plot(y+center_y, x+center_x, linewidth=2)
  plt.savefig(name)
  plt.close()

## Question 1

In [ ]:
original_1 = cv2.imread('1.JPG', 0)

filter(original_1, "mean", 3)
filter(original_1, "mean", 5)
filter(original_1, "gaussian", 3)
filter(original_1, "gaussian", 5)

In [ ]:
original_2 = cv2.imread('2.JPG', 0)

filter(original_2, "mean", 3)
filter(original_2, "mean", 5)
filter(original_2, "gaussian", 3)
filter(original_2, "gaussian", 5)

## Question 2

In [86]:
filtered_1 = cv2.imread("1_gaussian_3.png", 0)
filtered_2 = cv2.imread("2_gaussian_3.png", 0)

In [ ]:
# Image 1
histograms(filtered_1, "1_filtered")

hist_eq = cv2.equalizeHist(filtered_1)
histograms(hist_eq, "1_hist_eq")
print (psnr(filtered_1, hist_eq))

clahe = cv2.createCLAHE(clipLimit=4, tileGridSize=(8, 8))
clahe_eq = clahe.apply(filtered_1)
cv2.imwrite("1_eq.png", clahe_eq);
histograms(clahe_eq, "1_clahe_eq")
print (psnr(filtered_1, clahe_eq))

# Image 2
histograms(filtered_2, "2_filtered")

clahe_eq = clahe.apply(filtered_2)
cv2.imwrite("2_eq.png", clahe_eq);
histograms(clahe_eq, "2_clahe_eq")
print (psnr(filtered_2, clahe_eq))

In [96]:
# Image 1
eq = cv2.imread("1_eq.png", 0)

robert_edge_detector(eq, "1_robert")

sobel_edge_detector(eq, 3, "1_sobel_3")
sobel_edge_detector(eq, 5, "1_sobel_5")

canny_edge_detector(eq, 30, 100, "1_canny")
canny_edge_detector(eq, 70, 200, "1_canny")
canny_edge_detector(eq, 100, 200, "1_canny")

laplacian_edge_detector(eq, 3, "1_lap")
laplacian_edge_detector(eq, 5, "1_lap")

diff_of_gaussian(eq, 3,5, "1_dog")
diff_of_gaussian(eq, 3,7, "1_dog")
diff_of_gaussian(eq, 5,7, "1_dog")

# Image 2
eq = cv2.imread("2_eq.png", 0)

robert_edge_detector(eq, "2_robert")

sobel_edge_detector(eq, 3, "2_sobel_3")
sobel_edge_detector(eq, 5, "2_sobel_5")

canny_edge_detector(eq, 30, 100, "2_canny")
canny_edge_detector(eq, 80, 95, "2_canny")
canny_edge_detector(eq, 100, 200, "2_canny")

laplacian_edge_detector(eq, 3, "2_lap")
laplacian_edge_detector(eq, 5, "2_lap")

diff_of_gaussian(eq, 3,5, "2_dog")
diff_of_gaussian(eq, 3,7, "2_dog")
diff_of_gaussian(eq, 5,7, "2_dog")

## Question 3

In [125]:
edges_1 = cv2.imread("1_edges.png", 0)
hough_transform(edges_1, 1, 0.3, "1_hough_1")
hough_transform(edges_1, 2, 0.3, "1_hough_2")
hough_transform(edges_1, 3, 0.3, "1_hough_3")
hough_transform(edges_1, 4, 0.3, "1_hough_4")

In [126]:
edges_2 = cv2.imread("2_edges.png", 0)
hough_transform(edges_2, 1, 0.3, "2_hough_1")
hough_transform(edges_2, 2, 0.3, "2_hough_2")
hough_transform(edges_2, 3, 0.3, "2_hough_3")
hough_transform(edges_2, 4, 0.3, "2_hough_4")